In [ ]:
use_google_colab = False

# set a debug mode
debug = True

In [ ]:
if use_google_colab :
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/ColabNotebooks/sentiMentaL_tweets

In [ ]:
### IMPORTS AND CONSTANTS

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import utils.vocab_manip as vm
import utils.hashtag_dealing_methods as hdm
import utils.pooling as po
import utils.submission as sub

In [ ]:
### DATA LOADING
# Load the word embeddings
word_embeddings = np.load('embeddings.npy')
df_word_embeddings = pd.DataFrame(word_embeddings)
print('Word embeddings shape:', word_embeddings.shape)
df_word_embeddings.head(1)


In [ ]:
# Load the test set tweets
with open('twitter-datasets/test_data.txt', 'r', encoding='utf-8') as file:
    test_tweets = file.readlines()
    df_test_tweets = pd.DataFrame(test_tweets)

print('Test tweets:', df_test_tweets.head(1))

In [ ]:
# Load the vocabulary
with open('vocab_cut.txt', 'r', encoding='utf-8') as file:
    vocabulary = file.read().splitlines()

print('Vocabulary:', vocabulary[:20])

In [ ]:
# Create a dictionary to map words to their corresponding embeddings
word_to_embedding = {word: word_embeddings[i] for i, word in enumerate(vocabulary)}
df_word_embeddings = pd.DataFrame(word_to_embedding)
print('Word to embedding shape:', df_word_embeddings.shape)
df_word_embeddings.head(1)



In [ ]:
# Load positive training tweets and assign labels
with open('twitter-datasets/train_pos_full.txt', 'r', encoding='utf-8') as file:
    pos_tweets = file.readlines()

pos_labels = np.ones(len(pos_tweets), dtype=int)  # Assign label 1 for positive tweets
df_pos_tweets = pd.DataFrame(pos_tweets)

print('Positive tweets shape:', df_pos_tweets.shape)

df_pos_tweets.head(1)

In [ ]:
# Load negative training tweets and assign labels
with open('twitter-datasets/train_neg_full.txt', 'r', encoding='utf-8') as file:
    neg_tweets = file.readlines()

neg_labels = -1 * np.ones(len(neg_tweets), dtype=int)  # Assign label -1 for negative tweets
df_neg_tweets = pd.DataFrame(neg_tweets)

print('Negative tweets shape:', df_neg_tweets.shape)
df_neg_tweets.head(1)

In [ ]:

# Combine positive and negative tweets and labels
df_pos_tweets = pd.DataFrame(pos_tweets)
df_neg_tweets = pd.DataFrame(neg_tweets)

print("positive tweets shape:", df_pos_tweets.shape)
print("negative tweets shape:", df_neg_tweets.shape)
df_pos_tweets.head(1)

#### End of the loading

___________________________

In [ ]:
if debug:
    pos_tweets = pos_tweets[:10]
    neg_tweets = neg_tweets[:10]
    test_tweets = test_tweets[:10]
    #all_tweets = np.concatenate((train_tweets, test_tweets), axis=0)
    pos_labels = pos_labels[:10]
    neg_labels = neg_labels[:10]

In [ ]:
# Should be 2 times more tweets than labels
print('All tweets size:', len(pos_tweets)+ len(neg_tweets)+ len(test_tweets))
print('All labels size:', len(pos_labels)+ len(neg_labels))

In [ ]:
#reorder the vocabulary and the word embeddings according to the largest number of occurences first
vocabulary, word_embeddings = vm.reorder_vocabulary(pos_tweets, neg_tweets, test_tweets, vocabulary, word_embeddings, save_to_csv=True)

In [ ]:
# remove hashtags that are not in the vocabulary
pos_tweets, neg_tweets, test_tweets = hdm.process_tweets_hashtags(pos_tweets, neg_tweets, test_tweets, vocabulary)

In [ ]:
#save the words that are not in the vocabulary
vm.out_of_vocab_file(pos_tweets, neg_tweets, test_tweets, vocabulary)

In [ ]:
weights = vm.calculate_weights(pos_tweets, neg_tweets, vocabulary)
print('Weights shape:', weights.shape)

In [ ]:
print("Vocab shape:", len(vocabulary))

In [ ]:
### TRAINING THE LINEAR CLASSIFIER

pooling_method = "weigth" # "mean", "max", "tfidf", "weigth"

train_features, test_features = po.get_features(pooling_method, pos_tweets, neg_tweets, test_tweets, word_to_embedding, vocabulary)
# Split the data into training and validation sets
labels = np.concatenate((pos_labels, neg_labels), axis=0)


train_features = np.array(train_features)
labels = np.array(labels)
# Assuming train_features and labels are NumPy arrays
assert len(train_features) == len(labels), "Features and labels must be of the same length"

# Generate a permutation of indices
shuffled_indices = np.random.permutation(len(train_features))

# Apply the shuffled indices to both features and labels
shuffled_features = train_features[shuffled_indices]
shuffled_labels = labels[shuffled_indices]



X_train, X_val, y_train, y_val = train_test_split(train_features, labels, test_size=0.1, random_state=42)

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Validate
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy}")

In [ ]:
print('Test features:', len(test_features))

In [ ]:
### LINEAR CLASSIFIER PREDICTIONS

# Construct feature representations for test tweets

# Make predictions
y_test_pred = model.predict(test_features)

test_data_path = "twitter-datasets/test_data.txt"
ids_test = sub.get_test_ids(test_data_path)
print(y_test_pred)
y_pred = []
y_pred = y_test_pred
y_pred[y_pred <= 0] = -1
y_pred[y_pred > 0] = 1
sub.create_csv_submission(ids_test, y_pred, "submission.csv")